#pip install

In [1]:
!pip install pyunpack
!pip install patool
! pip install -Uq pyopenssl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 5.0 MB/s eta 0:00:00


In [2]:
!pip install av

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 33.5 MB/s eta 0:00:00


#download dataset

In [ ]:
!wget -P ~/.fastai/archive/ --no-check-certificate  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar

--2023-05-07 09:02:38--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘/root/.fastai/archive/UCF101.rar’

UCF101.rar           87%[================>   ]   5.67G  25.9MB/s    eta 32s    

#import things

In [ ]:
from pyunpack import Archive
import os
from tqdm import tqdm_notebook as tqdm
import glob
import fastai
from fastai.vision.all import *
from fastai.vision.core import show_images
import torch
import numpy as np
from PIL import Image
import av

In [ ]:
#unrat unzip like
Archive('/root/.fastai/archive/UCF101.rar').extractall('/content/')

In [ ]:
count=0
video_paths=[]
for a_class in tqdm(os.listdir("/content/UCF-101")):
  count+=len(os.listdir(f"/content/UCF-101/{a_class}"))
  for path in os.listdir(f"/content/UCF-101/{a_class}"):
    video_paths.append(f"/content/UCF-101/{a_class}/{path}")
print(count/101)

In [ ]:
"/content/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi"

In [ ]:
def extract_frames(video_path):
    video = av.open(str(video_path))
    for frame in video.decode(0):
        yield frame.to_image()

##example

In [ ]:
frames = list(extract_frames("/content/UCF-101/Biking/v_Biking_g01_c01.avi"))
print(np.array(frames[0]).shape)
frames[0]

In [ ]:
len(frames),frames[0]

In [ ]:
#from fastai.vision.core import show_images
show_images(frames,nrows=10, ncols=10)

##ds,dl

In [ ]:
instance_paths=glob.glob("/content/UCF-101/*/*",recursive=False)

In [ ]:
instance_paths

In [ ]:
class ImageTuple(fastuple):
    "A tuple of PILImages"
    def show(self, ctx=None, **kwargs): 
        n = len(self)
        img0, img1, img2= self[0], self[n//2], self[n-1]
        if not isinstance(img1, Tensor):
            t0, t1,t2 = tensor(img0), tensor(img1),tensor(img2)
            t0, t1,t2 = t0.permute(2,0,1), t1.permute(2,0,1),t2.permute(2,0,1)
        else: t0, t1,t2 = img0, img1,img2
        return show_image(torch.cat([t0,t1,t2], dim=2), ctx=ctx, **kwargs)

In [ ]:
img_sample=ImageTuple(PILImage.create(fn) for fn in frames).show()
img_sample.figure

In [ ]:
class ImageTupleTfm(Transform):
    def __init__(self, seq_len=20):
        store_attr()
    def encodes(self, path):
        frames = list(extract_frames(path))
        n_frames =len(frames)
        s = slice(0, min(self.seq_len, n_frames))
        return ImageTuple(tuple(PILImage.create(f) for f in frames[s]))

In [ ]:
tfm = ImageTupleTfm(seq_len=5)
biking_instance = instance_paths[0]
biking_instance

In [ ]:
tfm(biking_instance).show()

In [ ]:
splits = RandomSplitter()(instance_paths)
ds = Datasets(instance_paths, tfms=[[ImageTupleTfm(5)], [parent_label, Categorize]], splits=splits)

In [ ]:
len(ds)

In [ ]:
ds.tfms

In [ ]:
dls = ds.dataloaders(bs=24, after_item=[Resize(128), ToTensor], 
                      after_batch=[IntToFloatTensor, Normalize.from_stats(*imagenet_stats)])

In [ ]:
x,y = dls.one_batch()

In [ ]:
x

In [ ]:
y

In [ ]:
dls.loss_func

In [ ]:
dls.show_batch()

#model

##simple model

In [ ]:
class SimpleModel(Module):
    def __init__(self, arch=resnet34(), n_out=101):
        self.encoder = TimeDistributed(create_body(arch, pretrained=True))
        self.head = TimeDistributed(create_head(512, 101))
    def forward(self, x):
        x = torch.stack(x, dim=1)
        return self.head(self.encoder(x)).mean(dim=1)
def simple_splitter(model): return [params(model.encoder), params(model.head)]

In [ ]:
simple_model = SimpleModel().cuda()

In [ ]:
learn = Learner(dls, simple_model, metrics=[accuracy], splitter=simple_splitter).to_fp16()

In [ ]:
learn.fine_tune(3,1e-3, freeze_epochs=2)

##resnet RNN

In [ ]:
arch2 = resnet34()
encoder = nn.Sequential(create_body(arch2, pretrained=True), nn.AdaptiveAvgPool2d(1), Flatten()).cuda()

In [ ]:
encoder(x[0]).shape

In [ ]:
tencoder = TimeDistributed(encoder)
tencoder(torch.stack(x, dim=1)).shape

In [ ]:
class RNNModel(Module):
    def __init__(self, arch=resnet34, n_out=101, num_rnn_layers=1):
        self.encoder = TimeDistributed(nn.Sequential(create_body(arch, pretrained=True), nn.AdaptiveAvgPool2d(1), Flatten()))
        self.rnn = nn.LSTM(512, 512, num_layers=num_rnn_layers, batch_first=True)
        self.head = LinBnDrop(num_rnn_layers*512, n_out)
    def forward(self, x):
        x = torch.stack(x, dim=1)
        x = self.encoder(x)
        bs = x.shape[0]
        _, (h, _) = self.rnn(x)
        return self.head(h.view(bs,-1))

In [ ]:
def rnnmodel_splitter(model):
    return [params(model.encoder), params(model.rnn)+params(model.head)]

In [ ]:
RES_rnn = RNNModel().cuda()

In [ ]:
learn2 = Learner(dls,RES_rnn, metrics=[accuracy], splitter=rnnmodel_splitter).to_fp16()

In [ ]:
learn2.fine_tune(5, 5e-3)

In [ ]:
learn2.show_results()

#time transformers

In [ ]:
!pip install -Uq timesformer-pytorch

In [ ]:
from timesformer_pytorch import TimeSformer
class MyTimeSformer(TimeSformer):
    def forward(self, x):
        x = torch.stack(x, dim=1)
        return super().forward(x)

In [ ]:
timesformer = MyTimeSformer(
    dim = 128,
    image_size = 128,
    patch_size = 16,
    num_frames = 5,
    num_classes = 101,
    depth = 12,
    heads = 8,
    dim_head =  64,
    attn_dropout = 0.1,
    ff_dropout = 0.1
).cuda()

In [ ]:
learn_tf= Learner(dls, timesformer, metrics=[accuracy]).to_fp16()

In [ ]:
learn_tf.fit_one_cycle(12, 5e-4)

In [ ]:
learn_tf.show_results()